In [30]:
# importamos las liberias necesarias para trabajar con python con spark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, IntegerType, FloatType, DateType, BooleanType, DoubleType
from pyspark.sql.functions import *

# creamos la sesion de spark con 4 cores y 4 gigas de memoria mas nombre de la aplicacion
spark = SparkSession.builder.master("local[4]").appName("carga_de_datos").getOrCreate()

# creamos variable con la ruta del archivos
capaLimpieza = "../datalake/capaLimpieza/peliculas.parquet"

In [36]:
# cargamos los datos en un dataframe
df = spark.read.parquet(capaLimpieza).persist()

In [37]:
# miramos la columa title el numero de registros por title y solo mostramos los que son unicos
len(df.groupBy("title").agg(count("title").alias("num_registros")).where("num_registros == 1").collect())

5866

In [38]:
# filtramos los registros repetidos en la columna title
df = df.dropDuplicates(["title"])

df.count()

5866

In [39]:
# filtramos los registros repitidos en la columna poster
df = df.dropDuplicates(["poster"])

df.count()

5866

In [43]:
# contar los registros repitidos en la columna id
df.dropDuplicates(["id"]).count()

df.select("id").distinct().count()

df.select("id").show()

+------+
|    id|
+------+
|356895|
|321666|
| 46564|
|329289|
| 48686|
|264170|
| 61416|
|286657|
| 10800|
|224951|
| 15658|
|380734|
|213417|
|209274|
| 12117|
| 84305|
| 85872|
| 15138|
|413998|
| 96534|
+------+
only showing top 20 rows



In [35]:
# escribimos el dataframe en formato parquet
df.write.mode("overwrite").parquet(capaLimpieza)

# liberamos memoria
df.unpersist()


DataFrame[adult: boolean, genres: string, id: int, imdb_id: string, title: string, overview: string, release_date: string, runtime: double, cast: string, crew: string, keywords: string, poster: string]

In [29]:
# cerramos la sesion de spark
spark.stop()